In [1]:
!pip install torch
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 1.2MB 3.9MB/s 
     |████████████████████████████████| 1.9MB 5.2MB/s 
     |████████████████████████████████| 3.2MB 24.3MB/s 
     |████████████████████████████████| 890kB 38.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=e8717fd6d41514508fcd96967f61e95c7945dc66ac15cf2312ca1c354389259b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('ramsrigouthamg/t5_paraphraser')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd


device  cuda


In [4]:
class ParaphraseDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, max_len=512):
        self.path = os.path.join(data_dir, type_path + '.csv')

        self.source_column = "#1 String"
        self.target_column = "#2 String"
        self.data = pd.read_csv(self.path)

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in range(len(self.data)):
            input_, target = self.data.loc[idx, self.source_column], self.data.loc[idx, self.target_column]

            input_ = "paraphrase: "+ input_ + ' </s>'
            target = target + " </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt", truncation='longest_first'
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt", truncation='longest_first'
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [5]:
train_dataset = ParaphraseDataset(tokenizer, 'data', 'ms_data', 128)
test_dataset = ParaphraseDataset(tokenizer, 'data', 'ms_data_test', 128)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:186: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


In [28]:
from transformers import Trainer, TrainingArguments


def get_dataset(tokenizer, type_path, args):
    return ParaphraseDataset(tokenizer=tokenizer, data_dir='data', type_path=type_path,  max_len=128)

train_dataset = get_dataset(tokenizer=tokenizer, type_path="ms_all", args={})
train_dataloader = DataLoader(train_dataset, batch_size=16, drop_last=True, shuffle=True)

test_dataset = get_dataset(tokenizer=tokenizer, type_path="ms_data_test", args={})
test_dataloader = DataLoader(test_dataset, batch_size=16, drop_last=True, shuffle=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:186: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


In [29]:
from torch.utils.data import DataLoader
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(device)

model.to(device)
for param in model.base_model.parameters():
    param.requires_grad = False
model.train()

train_loader = train_dataloader

optim = AdamW(model.parameters(), lr=5e-5)
torch.cuda.empty_cache()

cuda


In [34]:
len(train_dataloader)

243

In [51]:
for epoch in range(5):
    c = 0
    for batch in train_loader:
        if c%100 == 0:
            print(f'{c} steps Done')
        c+=1
        optim.zero_grad()
        input_ids = batch['source_ids'].to(device)
        attention_mask = batch['source_mask'].to(device)
        decoder_inputs  = batch['target_ids'].to(device)
        decoder_attention_mask = batch['target_mask'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask,decoder_input_ids = decoder_inputs, decoder_attention_mask = decoder_attention_mask)
        loss = outputs[0]
#         loss.sum().backward()
        optim.step()
    print(f'==================== Epoch {epoch} Finished ===========================')

# model.eval()

0 steps Done
100 steps Done
200 steps Done
==================== Epoch 0 Finished ===========================
0 steps Done
100 steps Done
200 steps Done
==================== Epoch 1 Finished ===========================
0 steps Done
100 steps Done
200 steps Done
==================== Epoch 2 Finished ===========================
0 steps Done
100 steps Done
200 steps Done
==================== Epoch 3 Finished ===========================
0 steps Done
100 steps Done
200 steps Done
==================== Epoch 4 Finished ===========================


In [1]:
 sentence = "The organisers of a vigil for Sarah Everard are pursuing legal action after claiming the Metropolitan Police reversed their position on allowing the event to go ahead."

In [50]:
text =  sentence


max_len = 64

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    temperature=0.98,
    early_stopping=True,
    num_return_sequences=1,
    no_repeat_ngram_size = 3
)


print ("\nOriginal Sentence ::")
print (sentence)
print ("\n")
print ("Paraphrased Sentence :: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2155: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,



Original Sentence ::
The organisers of a vigil for Sarah Everard are pursuing legal action after claiming the Metropolitan Police reversed their position on allowing the event to go ahead.


Paraphrased Sentence :: 
0: Activists have filed criminal action against the Metropolitan Police for having stopped vigil for Sarah Everard at her home in a noisy corner allegedly called by an armed police stoplight. The alleged involved were treated as a child and the Police had no legal track record, they have not been informed and have not received any evidence supporting their case.
